<a href="https://colab.research.google.com/github/andy311p/Udemy_deep_learning_pytorch/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

train_dataset = dsets.MNIST(root='./data', 
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='./data', 
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

In [ ]:
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset)/batch_size))

#iterable object
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [ ]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel,self).__init__()
    
    #Convolution 1
    #use 16 kernels of depth 1 and size 5*5
    self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
    self.relu1 = nn.ReLU()
    #Max Pool 1
    #downsample 28*28 -> 14*14
    self.maxpool1 = nn.MaxPool2d(kernel_size=2)

    #Convolution 2
    #use 32 kernels of depth 16 and size 5*5
    self.cnn2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
    self.relu2 = nn.ReLU()
    #Max Pool 2
    #downsample 14*14 -> 7*7
    self.maxpool2 = nn.MaxPool2d(kernel_size=2)
    #self.avgpool2 = nn.AvgPool2d(kernel_size=2)# can also be changed to Average pooling
    
    #Linear (Fully Connected)
    self.fc1 = nn.Linear(32 * 7 * 7 ,10)

  def forward(self,x):
    out = self.cnn1(x)
    out = self.relu1(out)
    out = self.maxpool1(out)
    out = self.cnn2(out)
    out = self.relu2(out)
    out = self.maxpool2(out)

    #reshaping the out to fit a linear layer. technical step
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    return out  

In [ ]:
#image size
#input_dim = 28*28
#hidden_dim = 100
#num of classes(digits 0-9)
#output_dim=10

model = CNNModel()
#computes the softmax automatically
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
iter = 0
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_loader):
    #images = Variable(images.view(-1,28*28))
    images = Variable(images) #no need to reshape since CNN can take images directly
    labels = Variable(labels)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    iter += 1
    #every 500 iteration calculate accuracy
    if iter % 500 == 0:
      correct = 0.0
      total = 0.0
      #calc accuracy on test data
      for images, labels in test_loader:
        images = Variable(images)
        outputs = model(images)
        _, predictions = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predictions == labels).sum()

      accuracy = 100 * correct / total
      print('iteration: {}. loss: {} Accuracy: {}'.format(iter, loss.data,accuracy)) 


iteration: 500. loss: 0.24409785866737366 Accuracy: 93.33999633789062
iteration: 1000. loss: 0.28489574790000916 Accuracy: 93.80000305175781
iteration: 1500. loss: 0.0944928526878357 Accuracy: 95.75
iteration: 2000. loss: 0.11147098243236542 Accuracy: 96.37000274658203
iteration: 2500. loss: 0.09958186745643616 Accuracy: 96.73999786376953
iteration: 3000. loss: 0.06651869416236877 Accuracy: 96.93000030517578
